### Bootstrapping the bounding boxes:
In this step we will use the finetuned model to generate new bounding boxes on the data that the model has not seen. The correctly predicted boxes will be used to further finetune the model and the incorrect boxes will be used to manually draw the bounding boxes in the images.

Steps:
1. Get the species and count from the preidcted bounding boxes (data in the folder - training_demo/Predictions/)
-- This is a TFRecord 

2. The LILA dataset does not have information on count and so I will use the concensus data for this information

In [1]:
#import pandas as pd
import csv, os, sys, io
import operator
import tensorflow as tf
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
sys.path.append('/home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/data/')
from utils import dataset_util
#Added this to handle the truncation error while decoding the jpeg
from PIL import ImageFile
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

pd.options.display.max_columns = 100

In [2]:
Project_filepath = "/home/ubuntu/data/tensorflow/my_workspace/training_demo/"

In [3]:
with open('../data/LILA/SnapshotSerengeti.json', 'r') as f:
    dict1 = json.load(f)

# read the bounding box annotations and the image information to dataframes
df_annotations = pd.DataFrame(dict1['annotations'])
# Removing the unknown category from the dataset
df_annotations = df_annotations.loc[df_annotations['category_id']!=1000]
print(df_annotations.shape)
df_images = pd.DataFrame(dict1['images'])
print(df_images.shape)
df_categories = pd.DataFrame(dict1['categories'])
print(df_categories.shape)

(3241246, 3)
(3198737, 10)
(49, 2)


In [4]:
df_annotations.head()

,category_id,id,image_id
0,1,94cdea27-ad6d-11e8-88e9-000d3a062ba9,S1/B04/B04_R1/S1_B04_R1_PICT0001
1,1,94cdea29-ad6d-11e8-88e9-000d3a062ba9,S1/B04/B04_R1/S1_B04_R1_PICT0002
2,0,94cdea2b-ad6d-11e8-88e9-000d3a062ba9,S1/B04/B04_R1/S1_B04_R1_PICT0003
3,0,94cdea2d-ad6d-11e8-88e9-000d3a062ba9,S1/B04/B04_R1/S1_B04_R1_PICT0004
4,0,94cdea2f-ad6d-11e8-88e9-000d3a062ba9,S1/B04/B04_R1/S1_B04_R1_PICT0005


In [5]:
df_images.head()

,datetime,file_name,frame_num,height,id,location,season,seq_id,seq_num_frames,width
0,2010-07-18 16:26:14,S1/B04/B04_R1/S1_B04_R1_PICT0001.JPG,0,1536,S1/B04/B04_R1/S1_B04_R1_PICT0001,B04,S1,ASG0002kjh,1,2048
1,2010-07-18 16:26:30,S1/B04/B04_R1/S1_B04_R1_PICT0002.JPG,0,1536,S1/B04/B04_R1/S1_B04_R1_PICT0002,B04,S1,ASG0002kji,1,2048
2,NaN,S1/B04/B04_R1/S1_B04_R1_PICT0003.JPG,0,1536,S1/B04/B04_R1/S1_B04_R1_PICT0003,B04,S1,ASG0002kjj,1,2048
3,NaN,S1/B04/B04_R1/S1_B04_R1_PICT0004.JPG,0,1536,S1/B04/B04_R1/S1_B04_R1_PICT0004,B04,S1,ASG0002kjk,1,2048
4,NaN,S1/B04/B04_R1/S1_B04_R1_PICT0005.JPG,0,1536,S1/B04/B04_R1/S1_B04_R1_PICT0005,B04,S1,ASG0002kjl,1,2048


In [6]:
df_categories.head()

,id,name
0,0,empty
1,1,human
2,2,gazelleGrants
3,3,reedbuck
4,4,dikDik


In [10]:
# 2.
# Join the tables df_annotations and df_images
df_bb_meta = pd.merge(left=df_annotations, 
                      right=df_images, 
                      how='left',
                      left_on=df_annotations['image_id'], 
                      right_on=df_images['id'])
df_bb_meta = df_bb_meta.drop(columns=['key_0', 'id_y'])

df_bb_meta = pd.merge(left=df_bb_meta, 
                      right=df_categories, 
                      how='left',
                      left_on=df_bb_meta['category_id'], 
                      right_on=df_categories['id'])
df_bb_meta = df_bb_meta.drop(columns=['key_0', 'id'])
print("Shape before removing overlapping images with schneider's data: ", df_bb_meta.shape)

# Removing the overlapping images with Schneider's data
# df_bb_meta = df_bb_meta[~df_bb_meta['seq_id'].isin(schneider_events)]
# print("Shape after removing overlapping images with schneider's data: ", df_bb_meta.shape)

# 1.
# Removing the event that have been used for training the model
# event_in_training = list(df_bb_meta[df_bb_meta['image_id'].isin(l)]['seq_id'])
# df_bb_meta = df_bb_meta[~df_bb_meta['seq_id'].isin(event_in_training)]
# print("Shape after removing images used for training the model: ", df_bb_meta.shape)

# 2.
# Removing humans from the data
# df_bb_meta = df_bb_meta.loc[~df_bb_meta['category_id'].isin([0, 1])]
# print("Shape after removing human and empty images: ", df_bb_meta.shape)
# Converting the dataframe to th dictionary for convenience
# dict_bb_meta = df_bb_meta.to_dict('records')

Shape before removing overlapping images with schneider's data:  (3241246, 13)


In [11]:
df_bb_meta.head(n = 4)

,category_id,id_x,image_id,datetime,file_name,frame_num,height,location,season,seq_id,seq_num_frames,width,name
0,1,94cdea27-ad6d-11e8-88e9-000d3a062ba9,S1/B04/B04_R1/S1_B04_R1_PICT0001,2010-07-18 16:26:14,S1/B04/B04_R1/S1_B04_R1_PICT0001.JPG,0,1536,B04,S1,ASG0002kjh,1,2048,human
1,1,94cdea29-ad6d-11e8-88e9-000d3a062ba9,S1/B04/B04_R1/S1_B04_R1_PICT0002,2010-07-18 16:26:30,S1/B04/B04_R1/S1_B04_R1_PICT0002.JPG,0,1536,B04,S1,ASG0002kji,1,2048,human
2,0,94cdea2b-ad6d-11e8-88e9-000d3a062ba9,S1/B04/B04_R1/S1_B04_R1_PICT0003,NaN,S1/B04/B04_R1/S1_B04_R1_PICT0003.JPG,0,1536,B04,S1,ASG0002kjj,1,2048,empty
3,0,94cdea2d-ad6d-11e8-88e9-000d3a062ba9,S1/B04/B04_R1/S1_B04_R1_PICT0004,NaN,S1/B04/B04_R1/S1_B04_R1_PICT0004.JPG,0,1536,B04,S1,ASG0002kjk,1,2048,empty


In [12]:
df_concensus = pd.read_csv((Project_filepath + 'Data/consensus_data.csv'))
df_concensus.head(n = 2)

,CaptureEventID,NumImages,DateTime,SiteID,LocationX,LocationY,NumSpecies,Species,Count,Standing,Resting,Moving,Eating,Interacting,Babies,NumClassifications,NumVotes,NumBlanks,Evenness
0,ASG0002kjh,1,2010-07-18 16:26:14,B04,695238,9729497,1,human,1,0.625000,0.062500,0.0,0.000000,0.500000,0.0,17,16,1,0.0
1,ASG0002kji,1,2010-07-18 16:26:30,B04,695238,9729497,1,human,2,0.095238,0.619048,0.0,0.047619,0.333333,0.0,23,21,2,0.0


In [14]:
target = list(df_concensus['CaptureEventID'])
print(len(list(df_concensus['CaptureEventID'])))
print(len(set(df_concensus['CaptureEventID'])))

334671
319915


In [15]:
count_dict={}
for event in target:
    if event not in count_dict:
        count_dict[event] = 1
    else:
        count_dict[event] = count_dict[event] + 1

In [16]:
duplicates = {k : v for k,v in count_dict.items() if v > 3}
duplicates

{'ASG000q0hl': 4}

In [17]:
df_concensus[df_concensus['CaptureEventID'] == 'ASG000q0hl']

,CaptureEventID,NumImages,DateTime,SiteID,LocationX,LocationY,NumSpecies,Species,Count,Standing,Resting,Moving,Eating,Interacting,Babies,NumClassifications,NumVotes,NumBlanks,Evenness
259334,ASG000q0hl,3,2012-10-11 15:33:35,I01,710924,9735643,4,zebra,1,0.000,0.0,0.958333,0.041667,0.0,0.000000,25,24,0,0.774047
259335,ASG000q0hl,3,2012-10-11 15:33:35,I01,710924,9735643,4,warthog,4,0.000,0.0,1.000000,0.000000,0.0,0.695652,25,23,0,0.774047
259336,ASG000q0hl,3,2012-10-11 15:33:35,I01,710924,9735643,4,topi,1,0.000,0.0,1.000000,0.000000,0.0,0.000000,25,16,0,0.774047
259337,ASG000q0hl,3,2012-10-11 15:33:35,I01,710924,9735643,4,gazelleGrants,2,0.125,0.0,0.875000,0.000000,0.0,0.000000,25,8,0,0.774047


Bring in the info about the count to the all_image dataframe

In [12]:
df_image_wc = pd.merge(left=df_bb_meta,
        right=df_concensus,
        how='left',
        left_on=df_images['seq_id'],
        right_on=df_concensus['CaptureEventID'])
df_image_wc = df_image_wc.drop(columns=['key_0', 'CaptureEventID'])

KeyError: 0          ASG0002kjh
1          ASG0002kji
2          ASG0002kjj
3          ASG0002kjk
4          ASG0002kjl
5          ASG0002kjm
6          ASG0002kjn
7          ASG0002kjo
8          ASG0002kjp
9          ASG0002kjq
10         ASG0002kjr
11         ASG0002kjs
12         ASG0002kjt
13         ASG0002kju
14         ASG0002kjv
15         ASG0002kjw
16         ASG0002kjx
17         ASG0002kjy
18         ASG0002kjz
19         ASG0002kk0
20         ASG0002kk1
21         ASG0002kk2
22         ASG0002kk3
23         ASG0002kk4
24         ASG0002kk4
25         ASG0002kk4
26         ASG0002kk5
27         ASG0002kk5
28         ASG0002kk5
29         ASG0002kk6
              ...    
3198707    ASG0011wg7
3198708    ASG0011wg7
3198709    ASG0011wg9
3198710    ASG0011wg9
3198711    ASG0011wg9
3198712    ASG0011wga
3198713    ASG0011wga
3198714    ASG0011wga
3198715    ASG0011wgb
3198716    ASG0011wgb
3198717    ASG0011wgb
3198718    ASG0011wgc
3198719    ASG0011wgc
3198720    ASG0011wgc


In [22]:
print(df_image_wc.shape[0] - df_images.shape[0])
df_image_wc.head()

42509


,datetime,file_name,frame_num,height,id,location,season,seq_id,seq_num_frames,width,NumImages,DateTime,SiteID,LocationX,LocationY,NumSpecies,Species,Count,Standing,Resting,Moving,Eating,Interacting,Babies,NumClassifications,NumVotes,NumBlanks,Evenness
0,2010-07-18 16:26:14,S1/B04/B04_R1/S1_B04_R1_PICT0001.JPG,0,1536,S1/B04/B04_R1/S1_B04_R1_PICT0001,B04,S1,ASG0002kjh,1,2048,1.0,2010-07-18 16:26:14,B04,695238.0,9729497.0,1.0,human,1,0.625000,0.062500,0.0,0.000000,0.500000,0.0,17.0,16.0,1.0,0.0
1,2010-07-18 16:26:30,S1/B04/B04_R1/S1_B04_R1_PICT0002.JPG,0,1536,S1/B04/B04_R1/S1_B04_R1_PICT0002,B04,S1,ASG0002kji,1,2048,1.0,2010-07-18 16:26:30,B04,695238.0,9729497.0,1.0,human,2,0.095238,0.619048,0.0,0.047619,0.333333,0.0,23.0,21.0,2.0,0.0
2,NaN,S1/B04/B04_R1/S1_B04_R1_PICT0003.JPG,0,1536,S1/B04/B04_R1/S1_B04_R1_PICT0003,B04,S1,ASG0002kjj,1,2048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,S1/B04/B04_R1/S1_B04_R1_PICT0004.JPG,0,1536,S1/B04/B04_R1/S1_B04_R1_PICT0004,B04,S1,ASG0002kjk,1,2048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,S1/B04/B04_R1/S1_B04_R1_PICT0005.JPG,0,1536,S1/B04/B04_R1/S1_B04_R1_PICT0005,B04,S1,ASG0002kjl,1,2048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
l1 = list(set(df_images['seq_id']))
l2 = list(set(df_concensus['CaptureEventID']))
# [l1[i] for i in range(len(l1)) if l1[i] in l2]
len(list(set(df_images['seq_id']) & set(df_concensus['CaptureEventID'])))

319908

In [35]:
3198737 - df_concensus.shape[0]

2864066

Run through the TFRecord and check the count and list of animals. logic: <br>
1. Predictions Species, count per image
2. Ground truth Species, count per image from Concensus data or data prepped by marco

In [3]:
def decode_record(serialized_example):
    context_features = {
                        'image/filename': tf.FixedLenFeature([], tf.string),
                        'image/encoded': tf.FixedLenFeature([], tf.string),
                        'image/format': tf.FixedLenFeature([], tf.string),
                        "image/detection/bbox/xmin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/xmax" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymax" : tf.VarLenFeature(tf.float32),
                        "image/detection/label" : tf.VarLenFeature(tf.int64),
                        "image/detection/score" : tf.VarLenFeature(tf.float32)
                    }


    context, sequence = tf.parse_single_sequence_example(serialized=serialized_example,
                                              context_features=context_features,
#                                               sequence_features=sequence_features,
                                              example_name=None,
                                              name=None)

    return ({k: v for k, v in context.items()},{k: v for k, v in sequence.items()})

In [80]:
filename_list = ['/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_s01_s06-0-10000.record']
batch_size = 2
dataset = tf.data.Dataset.from_tensor_slices(filename_list)
dataset = tf.data.TFRecordDataset(dataset)
# dataset = dataset.shuffle(buffer_size=batch_size)
# dataset = dataset.repeat(1)
dataset = dataset.map(lambda x: decode_record(serialized_example=x)).batch(batch_size)

iterator = dataset.make_one_shot_iterator() # create the iterator
batch_data = iterator.get_next()

# print(dataset.output_types)
# print(dataset.output_shapes) 

In [81]:
# batch_data = iterator.get_next()
with tf.Session() as sess:
    (context, sequence) = sess.run(batch_data)
    img = context['image/encoded']
    # Detection 
    filename = context['image/filename']
    xmin_d = (context['image/detection/bbox/xmin'])
    ymin_d = (context['image/detection/bbox/ymin'])
    xmax_d = (context['image/detection/bbox/xmax'])
    ymax_d = (context['image/detection/bbox/ymax'])
    label_d = (context['image/detection/label'])
    score = (context['image/detection/score'])
    
    score_threshold = 0.5
    i = 0
    for rec_i in range(len(img)):
        encoded_jpg_io = io.BytesIO(img[rec_i])
        image = Image.open(encoded_jpg_io)
        width, height = image.size
        
        xmins_d, ymins_d, xmaxs_d, ymaxs_d, labels_d, scores = [], [], [], [], [], []
        
        for j in range(rec_i*75, (rec_i+1)*75):
            if xmin_d[0][j][0] == rec_i and score[1][j]>=score_threshold:
                xmins_d.append(int(xmin_d[1][j]*width))
                ymins_d.append(int(ymin_d[1][j]*height))
                xmaxs_d.append(int(xmax_d[1][j]*width))
                ymaxs_d.append(int(ymax_d[1][j]*height))
                labels_d.append(label_d[1][j])
                scores.append(round(score[1][j]*100, 1))
                
        print(rec_i, filename[rec_i], labels_d, scores)
            
    

0 b'S1/B04/B04_R1/S1_B04_R1_PICT0009' [5] [99.1]
1 b'S1/B04/B04_R1/S1_B04_R1_PICT0012' [5] [99.2]


In [84]:
db_dict = {}
with open('../data/LILA/db_export_season_all.csv', 'r') as csvfile:
    csvdata = csv.reader(csvfile, delimiter=',')
    header = next(csvdata)
    col_mapper = {k: i for i, k in enumerate(header)}
    for i, row in enumerate(csvdata):
        filename = row[col_mapper['PathFilename']]
        if filename not in db_dict.keys():
            db_dict[filename] = []
        db_dict[filename].append({k: row[v] for k, v in col_mapper.items() })
        if i%50000 == 0:
            print("performing iteration : %s" % i)

performing iteration : 0
performing iteration : 50000
performing iteration : 100000
performing iteration : 150000
performing iteration : 200000
performing iteration : 250000
performing iteration : 300000
performing iteration : 350000
performing iteration : 400000
performing iteration : 450000
performing iteration : 500000
performing iteration : 550000
performing iteration : 600000
performing iteration : 650000
performing iteration : 700000
performing iteration : 750000
performing iteration : 800000
performing iteration : 850000
performing iteration : 900000
performing iteration : 950000
performing iteration : 1000000
performing iteration : 1050000
performing iteration : 1100000
performing iteration : 1150000
performing iteration : 1200000
performing iteration : 1250000
performing iteration : 1300000
performing iteration : 1350000
performing iteration : 1400000
performing iteration : 1450000
performing iteration : 1500000
performing iteration : 1550000
performing iteration : 1600000
per

In [82]:
# db_dict

In [83]:
# pd.read_csv('../data/LILA/db_export_season_all.csv')